Nettoyage et préparation des données statiques

## Imports

In [ ]:
import polars as pl
import os
import re
import json

## Params

In [ ]:
# Chargement des paramètres du projet
with open('../params.json', 'r') as file :
    params = json.load(file)

DATASET, VERSION = params['dataset'], params['version'] # Nom du dataset et de la version
DATA_FOLD = params['data_folder'] # Localisation des données (entrée et sortie)


In [ ]:
# Localisation des fichiers
INPUT_FOLDER = f'{DATA_FOLD}/{VERSION}/1.raw_data/{DATASET}/'
OUTPUT_FOLDER = f'{DATA_FOLD}/{VERSION}/2.clean_data/{DATASET}/static/'

CENSUS_FILE = 'census/raw_census.xlsx'
IGS_FILE = 'igs/igs_all.csv'
OUTPUT_STATIC_FILE = 'clean_static_encounters.parquet'

## Callable

In [ ]:
def clean_percentage(value):
    # Probabilité de décès par IGS II en valeur décimale
    try:
        clean_value = float(value.replace('Mortalité prédite : ', '').replace('%', '').replace(',', '.')) / 100
        return clean_value
    except :
        return None

def clean_encounter(value):
    # Nettoyage des données sur la table des identifiants
    try:
        remove_letters = re.sub('\D', '', value)
        return remove_letters
    except:
        return None

def clean_igs(value):
    try:
        clean_value = float(value)
        return clean_value
    except:
        return None


## Import des datasets

In [ ]:
raw_data = pl.read_excel(INPUT_FOLDER + CENSUS_FILE, schema_overrides={"lifeTimeNumber": pl.Int64}) # Données statiques brutes
df_igs = pl.read_csv(INPUT_FOLDER + IGS_FILE)

In [ ]:
raw_data.head()

In [ ]:
if DATASET == 'mimic' :
    raw_data = raw_data.rename(
        # Renommage des variables Mimic pour correspondre aux noms sur le dataset CHU
                    {
                        'displaylabel': 'displayLabel',
                        'encounterid': 'encounterId',
                        'encounternumber': 'encounterNumber',
                        'lifetimenumber': 'lifeTimeNumber',
                        'dateofdeath' : 'dateOfDeath',
                        'isdeceased' : 'isDeceased',
                        'utcintime' : 'utcInTime',
                        'utcouttime' : 'utcOutTime',
                        'lengthofstay' : 'lengthOfStay',
                        'height' : 'taille',
                        'weight' : 'poids_admission'
                    }
                ).with_columns(
                    # Création de colonnes vides pour les données nominatives de Mimic
                    [
                        pl.lit(None).alias('adresse'),
                        pl.lit(None).alias('ville'),
                        pl.lit(None).alias('cp'),
                        pl.lit(None).alias('dateOfBirth'),
                        pl.lit(None).alias('lastName'),
                        pl.lit(None).alias('firstName')
                    ]
                )

if DATASET == 'chu' :
    raw_data = raw_data.with_columns(
        # Création de colonnes vides pour les données de taille / poids des données CHU
                    [
                        pl.lit(None).alias('taille'),
                        pl.lit(None).alias('poids_admission'),
                    ]
    )

In [ ]:
# Liste des unités de soins
raw_data.columns

## Data cleaning

### Separate mortality 

In [ ]:
df_insee = pl.read_csv(f'{DATA_FOLD}/{VERSION}/1.raw_data/chu/census/deces_insee.csv', ignore_errors=True)
df_mortality= (df_insee.filter(pl.col("upper") == 1)
    .cast(
        {"date_deces" : pl.Date},
        strict=False
    )
    .with_columns(
            pl.when(pl.col('DECES_PASTEL') == "NA")
            .then(pl.lit(None))
            .otherwise(pl.lit(True))
            .alias("deces_hosp")
)
    .sort(["NIP", "mean_score", "date_deces"], descending=[True, True, False])
    .unique("NIP", keep="first")
    .drop(
        ['id', 'probas_rf', 'probas_nn', 'upper','numero_acte_deces', 'mean_score', 'DECES_PASTEL' ]
        )
    .rename(
        {"NIP" : "lifeTimeNumber"}
    )
    )

df_mortality = raw_data.select('encounterId', 'dateOfDeath','isDeceased'
                ).filter((pl.col('isDeceased') == True) | (pl.col('dateOfDeath').is_not_null())
                         ).group_by('encounterId').agg(
                             pl.col('isDeceased').max().alias('isDeceased'),
                                pl.col('dateOfDeath').first().alias('dateOfDeath')
                ).cast(
                    {'encounterId' : pl.String}
                )

In [ ]:
df_mortality.filter(pl.col("deces_hosp") == False)

### Regroup by Encounter

In [ ]:
# Retrait des unités de chirurgie cardiaque dans les 2 datasets 
# Motif : Chirurgie majoritairement programmée, durée de séjours courte, mortalité faible et volume important de patients

# Retrait des unités de réanimation des grands brûlés, néphrologique et pédiatrique dans le dataset CHU
# Motif : Typologie de réanimation non présentes dans le dataset MIMIC
if DATASET == 'chu' :
    icu_units = [ 
                'PURPAN REA. POLY.',
                'IUC REA.', 
                'NEURO-CHIR REA', 
                'RANGUEIL REA. POLY.',
                'RANGUEIL DECHO. REA.', 
                'PURPAN DECHO. REA.', 
                'PURPAN SC. REA.', 
                'RANGUEIL SC. REA.',
                'IUC SC.',
                'C.C.V. REA',
                'GRANDS BRULES',
                'NEURO-CHIR USC'
                ]
    
elif DATASET == 'mimic' :
    icu_units = [
                'Neuro Surgical Intensive Care Unit (Neuro SICU)',
                'Surgical Intensive Care Unit (SICU)',
                'Neuro Stepdown',
                'Medical/Surgical Intensive Care Unit (MICU/SICU)',
                'Cardiac Vascular Intensive Care Unit (CVICU)', #retrait chirurgie CCV pour correspondre au dataset CHU
                'Neuro Intermediate',
                'Coronary Care Unit (CCU)',
                'Medical Intensive Care Unit (MICU)',
                'Trauma SICU (TSICU)'
                ]

In [ ]:
units_ipp = [ 
                'PURPAN REA. POLY.',
                'IUC REA.', 
                'NEURO-CHIR REA', 
                'RANGUEIL REA. POLY.',
                'RANGUEIL DECHO. REA.', 
                'PURPAN DECHO. REA.', 
                'PURPAN SC. REA.', 
                'RANGUEIL SC. REA.',
                'IUC SC.',
                'C.C.V. REA',
                'GRANDS BRULES',
                'NEURO-CHIR USC'
                
]

ipp_list = (raw_data
                .filter(
                    # Suppression des unités non désirées
                    pl.col('displayLabel').is_in(units_ipp),
                    # Suppression des patients mineurs lors de leur admission
                    (pl.col('age') >= 18)
                    )
                .with_columns(
                    # Nettoyrage des encounterId
                    pl.col('encounterId').cast(pl.String).map_elements(clean_encounter, return_dtype=pl.String).alias('encounterId')
                    )
                .sort(by=['encounterId', 'utcInTime'])
)

ipp_list.unique('lifeTimeNumber').select('lifeTimeNumber', 'utcInTime')#.write_csv('/data2/poette.m/dypo/v4/1.raw_data/chu/census/ipp_pour_matching.csv')

In [ ]:
encounter_df = (raw_data
                .filter(
                    # Suppression des unités non désirées
                    pl.col('displayLabel').is_in(icu_units),
                    # Suppression des patients mineurs lors de leur admission
                    (pl.col('age') >= 18)
                    )
                .with_columns(
                    # Nettoyrage des encounterId
                    pl.col('encounterId').cast(pl.String).map_elements(clean_encounter, return_dtype=pl.String).alias('encounterId')
                    )
                .sort(by=['encounterId', 'utcInTime'])
                .group_by(
                    ['encounterId',
                    'encounterNumber',
                    'lifeTimeNumber',
                    'lastName',
                    'firstName',
                    'gender',
                    'age',
                    'dateOfBirth',
                    'lengthOfStay']
                )
                .agg([
                # Récupération des dates d'entrée minimales et de sortie maximale pour chaque séjour
                # Séléction de la première unité de réanimation 
                    pl.col('utcInTime').min().alias('utcInTime'),
                    pl.col('utcOutTime').max().alias('utcOutTime'),
                    pl.col("displayLabel")
                    .filter((pl.col("displayLabel").is_not_null()))
                    .sort_by('lengthOfStay', descending=True)
                    .first()
                    .alias('unitLabel'),
                    pl.col('adresse').max().alias('adresse'),
                    pl.col('ville').max().alias('ville'),
                    pl.col('cp').max().alias('cp'),
                    pl.col('taille').first().alias('taille'),
                    pl.col('poids_admission').first().alias('poids_admission')
                ])
                .join(
                    # Jointure sur le dataset de mortalité
                    df_mortality, on='lifeTimeNumber', how='left'
                ).with_columns(
                    [
                    # Durée de séjour en heures (date de sortie max - date d'entrée min)
                        ((((pl.col('utcOutTime').sub(pl.col('utcInTime'))).dt.total_minutes())/60)).round(2).alias('los')
                    ]
                ).with_columns(
                    # Survenue d'un décès défini comme la présence d'une date de décès ou du statut'décès' concernant le mode de sortie
                    # Si ni l'un ni l'autre = survie
                    pl.col('date_deces').is_not_null().alias("isDeceased"),
                    pl.col('deces_hosp').is_not_null().alias("deces_hosp"),
                    (pl.col('date_deces')- pl.col('utcInTime')).dt.total_days().alias('deces_datediff')
                ).drop('lengthOfStay')

            )

In [ ]:
encounter_df.filter(pl.col('deces_datediff') > 30)

In [ ]:
encounter_df

## Extended demography

### Import datasets

In [ ]:
# Pour le Dataset CHU : ajout des motifs d'entrée et de sortie (en texte libre) et ajout du poids et de la taille
df_demo_extended = encounter_df
if DATASET == 'chu': 
    df_demo_extended = df_demo_extended.drop(['taille', 'poids_admission'])
    directory = INPUT_FOLDER + 'extended_demography/'
    for filename in os.listdir(directory):
        if filename.endswith(".parquet"): 
            df_extended_feature = pl.read_parquet(directory + filename)
            print(df_extended_feature.shape)
            feature = df_extended_feature.get_column('feature').to_list()[0]
            print(feature)
            df_extended_feature = df_extended_feature.sort(
                            'encounterId', 'utcChartTime'
                        ).unique(
                            subset=['encounterId'], keep='last'
                        )
            if feature in ['taille', 'poids_admission'] :
                df_extended_feature = df_extended_feature.rename(
                        {'valueNumber' : feature}
                )
            else:
                df_extended_feature = df_extended_feature.rename(
                        {'valueString' : feature}
                )
            df_extended_feature = df_extended_feature.select('encounterId', feature)
            df_demo_extended = df_demo_extended.join(
                            df_extended_feature, on='encounterId', how='left'
                        )

In [ ]:
df_demo_extended['unitLabel'].unique()

In [ ]:
if DATASET == 'chu': 
    ttt = pl.read_csv(INPUT_FOLDER + '/extended_demography/ttt.csv', separator=';').sort(by=['encounterId', 'storeTime']).unique('encounterId', keep='last')
    atcd = pl.read_csv(INPUT_FOLDER + '/extended_demography/atcd.csv', separator=';').sort(by=['encounterId', 'storeTime']).unique('encounterId', keep='last')
    ttt = ttt.select('encounterId', 'valueString').rename({'valueString' : 'traitements'}).cast({'encounterId' : pl.String})
    atcd = atcd.select('encounterId', 'valueString').rename({'valueString' : 'atcd'}).cast({'encounterId' : pl.String})
    df_demo_extended = df_demo_extended.join(ttt, on = 'encounterId', how='left').join(atcd, on = 'encounterId', how='left')

In [ ]:
df_demo_extended.join(ttt, on = 'encounterId', how='left').join(atcd, on = 'encounterId', how='left')

## IGS data

In [ ]:
# Dataset Mimic : ajout des données présentes dans l'IGS (mtype d'admission, score SAPS, probabilité de survie)
if DATASET == 'mimic' :
    df_igs_clean = (
            df_igs  
            .rename(
                {
                    'encounterid': 'encounterId'
                    ,'admissiontype_score' : 'admission_type'
                }
            )
            .sort('encounterId', 'sapsii', descending=[False, True])
            .unique('encounterId', keep='first')
            .with_columns(
                pl.when(pl.col('admission_type') == 8)
                    .then(pl.lit('Unscheduled Surgery'))
                .when(pl.col('admission_type') == 0)
                    .then(pl.lit('Scheduled Surgery'))
                .when(pl.col('admission_type') == 6)
                    .then(pl.lit('Medical'))
                .otherwise(None).alias('admission_type')
                )
            .cast({'encounterId': pl.String})
            .select('encounterId', 'admission_type', 'sapsii', 'sapsii_prob')
            )
    df_demo_extended = df_demo_extended.join(df_igs_clean, on='encounterId', how='left')

In [ ]:
df_igs

In [ ]:
# Dataset CHU : ajout des données présentes dans l'IGS (mtype d'admission, score SAPS, probabilité de survie)
# Seul le premier IGS est gardé si plusieurs présents
if DATASET == 'chu' : 
    df_igs_clean = (
                df_igs  
                .with_columns(
                        pl.col('encounterNumber').map_elements(clean_encounter, return_dtype=pl.String).alias('encounterNumber'),
                        pl.col('igsMort').map_elements(clean_percentage, return_dtype=pl.Float64).alias('sapsii_prob')
                        )
                .sort('encounterNumber', 'igsStoreTime')
                .unique('encounterNumber', keep='first')
                .with_columns(
                    pl.when(pl.col('igsTypeAdm') == 0)
                        .then(pl.lit('Medical'))
                    .when(pl.col('igsTypeAdm') == 2)
                        .then(pl.lit('Unscheduled Surgery'))
                    .when(pl.col('igsTypeAdm') == 1)
                        .then(pl.lit('Scheduled Surgery'))
                    .otherwise(None).alias('admission_type')
                    )
                .rename(
                    {
                    'igsTotal' : 'sapsii',
                    'igsGlgw' : 'score_glasgow'
                    }
                )
                .select('encounterNumber', 'admission_type', 'sapsii', 'sapsii_prob', 'score_glasgow')
                .cast(
                    {'encounterNumber' : pl.Int64}, 
                    strict = False
                )
                )
    
    df_demo_extended = df_demo_extended.join(df_igs_clean, on='encounterNumber', how='left')

In [ ]:
df_igs_clean

In [ ]:
df_demo_extended['unitLabel'].value_counts()

In [ ]:
df_demo_extended.head(5)

### Admission type (from IGS)

## Pseudonymisation

In [ ]:
df_demo_extended.columns

In [ ]:
# Définition des colonnes identifiantes
col_identifiantes = [
    'encounterNumber',
    'lifeTimeNumber',
    'lastName',
    'firstName',
    'dateOfBirth',
    'cp',
    'ville',
    'adresse',
    'utcInTime',
    'utcOutTime',
    'date_deces',
    'code_lieu_deces',
]

In [ ]:
# Retrait des colonnes identifiantes
# Attention dataset pseudonymisé non utilisable pour le traitement des séries temporelles car absence d'horodatage des entrées/sorties
# => Uniquement pour analyse démogrpahique
if DATASET == 'chu' : 
    df_indexed = df_demo_extended.with_row_index(offset=1)
    table_corr = df_indexed.select(col_identifiantes)
    df_pseudonymised = (
        df_indexed
            .with_columns(
                year_inTime = pl.col('utcInTime').dt.year()
            )
            .select(
                pl.exclude(col_identifiantes)
                )
        )
    df_pseudonymised.head(5)
    df_pseudonymised.head(5)

In [ ]:
df_pseudonymised

## Save dataset

In [ ]:
df_demo_extended.write_parquet(OUTPUT_FOLDER + OUTPUT_STATIC_FILE)

In [ ]:
df_demo_extended.shape

In [ ]:
print(OUTPUT_FOLDER + OUTPUT_STATIC_FILE)

In [ ]:
# Création d'une table de correspondance pour le dataset CHU
if DATASET == 'chu' : 
    table_corr.write_csv(OUTPUT_FOLDER + 'correlation_table.csv')
    df_pseudonymised.write_parquet(OUTPUT_FOLDER + 'clean_pseudonimysed_dataset.parquet')

In [ ]:
df_pseudonymised.head(5)